In [1]:
#import file
import pandas as pd
df = pd.read_csv('order_brush_order.csv')

In [2]:
df.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [3]:
df['event_time'] = pd.to_datetime(df['event_time'], format='%Y-%m-%d %H:%M:%S')

df.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [4]:
#Change file format to be groupe within every 1 hour
def get_newtime(column):
    y = column.year
    m = column.month
    d = column.day
    h = column.hour
    return '{y}-{m}-{d} {h}'.format(y=y, m=m, d=d, h=h)

df['new_time'] = df['event_time'].apply(get_newtime)

df.head()

,orderid,shopid,userid,event_time,new_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03,2019-12-27 0
1,31118059853484,156423439,46057927,2019-12-27 11:54:20,2019-12-27 11
2,31123355095755,173699291,67341739,2019-12-27 13:22:35,2019-12-27 13
3,31122059872723,63674025,149380322,2019-12-27 13:01:00,2019-12-27 13
4,31117075665123,127249066,149493217,2019-12-27 11:37:55,2019-12-27 11


In [5]:
df = df.drop('event_time', axis=1)

In [7]:
#group shopid and userid for every hour to look up for order brushing
new_df = df.groupby(['shopid','userid','new_time']).count()
new_df

orderid
shopid    userid    new_time              
10009     196962305 2019-12-27 3         1
10051     2854032   2019-12-27 19        1
          48600461  2019-12-29 1         1
10061     62464559  2019-12-31 2         1
          130633421 2019-12-30 21        1
...                                    ...
214949521 46269178  2019-12-31 20        1
214964814 200983383 2019-12-29 22        1
215175775 13688804  2019-12-31 9         1
          129266028 2019-12-31 14        1
215435223 79515106  2019-12-31 18        1

[215877 rows x 1 columns]

In [26]:
#shops are deemed to have conducted order brushing if their concentrate rate is greater than or equal to 3 at any instance
possible_brush = new_df[new_df['orderid']>=3].reset_index()
possible_brush

,shopid,userid,new_time,orderid
0,10159,214988798,2019-12-29 20,3
1,10287,198097381,2019-12-30 0,3
2,10287,214226569,2019-12-31 1,3
3,10402,77819,2019-12-28 14,3
4,10536,672345,2019-12-27 11,3
...,...,...,...,...
737,203531250,114282846,2019-12-29 17,3
738,204225676,198662175,2019-12-31 0,5
739,208696908,214111334,2019-12-30 17,3
740,210197928,52867898,2019-12-27 7,4


In [27]:
possible_brush.shopid.nunique()

296

In [40]:
#If there is more than 1 userid identified as suspicious, list all the userids separated by “&”, with the smaller numerical userid first.
userids = []
possible_brush.reset_index().groupby('shopid')['userid'].apply(lambda x: userids.append(x.values))

def concat_and(arr):
    res = '&'.join(str(x) for x in arr)
    return res

concat_userids = []
for element in userids:
    concat_userids.append(concat_and(element))

brush_df = pd.DataFrame({"shopid": possible_brush.reset_index()['shopid'].unique(), "userid": concat_userids})
brush_df

,shopid,userid
0,10159,214988798
1,10287,198097381&214226569
2,10402,77819
3,10536,672345&672345
4,29583,214495220&215275495
...,...,...
291,203531250,114282846
292,204225676,198662175
293,208696908,214111334
294,210197928,52867898


In [44]:
#If a shop is not deemed to have conducted order brushing, assign the value 0
default_df = pd.DataFrame({'shopid': df['shopid'].unique(), 'userid': 0})

In [46]:
#Submission
#Check each shop and determine whether it is deemed to have conducted order brushing. If a shop conducted order brushing, list the userid(s) that are identified as suspicious for the corresponding shopid.
#Two columns required: shopid ,userid
sol_df = pd.concat([default_df[~default_df.shopid.isin(brush_df.shopid)], brush_df])
sol_df

,shopid,userid
0,93950878,0
1,156423439,0
2,173699291,0
3,63674025,0
4,127249066,0
...,...,...
291,203531250,114282846
292,204225676,198662175
293,208696908,214111334
294,210197928,52867898


In [47]:
sol_df.to_csv("solution.csv", index=False)